# Instructions
The following code was designed to help select parameters for each channel to be used when running CellCounter.ipynb.  On a sample/composite image for which cells have been manually counted (instructions below) an iterative series of thresholds are applied before automatic counting is performed and the results for each threshold can then be compared to the manual counts. The goal is to select thresholds that yield the greatest consistency with manual counts.  Consistency is based upon the number of counted cells and their locations.   

*Note that this procedure must be performed separately for each channel to be counted.


### The data returned
This code will return a summary file with the following info for each threshold applied: the threshold applied, the Otsu threshold, the number of manually counted cells, the avg cell diameter specified by the user, the number of automatically counted cells, the number of automatically counted cells that correspond to the location of a single manually counted cell (Hits), and the average size of automatically counted cells (in pixels).  Additionally, two measures of accuracy are returned: the number of hits divided by the number of manually counted cells, and the number of hits divided by the number of automatically counted cells.

*The Otsu threshold defines a threshold that optimally separates background from signal (http://www.labbookpages.co.uk/software/imgProc/otsuThreshold.html).  This will not necessarily be the threshold that one chooses, but is good to use as a comparison.  One may pick a threshold that is more stringent than OTSU (ie, higher than OTSU), in order to select cells that are more prominent.  


### Choosing a threshold
An optimal threshold should:

1 - Identify the location of a large number of the manually counted cells.

2 - Have a relatively small number of automatically counted cells that are not manually counted (i.e. few false positives).

3 - Separate signal from background.  

The first criterion can be evaluated by examining the output variable Hits/Manually Counted Cells.  This is the proportion of manually counted cells that the counter identifies the location of.  In practice, it is easy to get values above .8-.90.  However, it can be the case that with a given threshold the counter correctly marks 100% of manually counted cells but also marks many more spots where no cells are present. Hence the second criterion.

The second criterion is harder to judge but can be evaluated by examining the output variable Hits/Automatically Counted Cells.  That is, what is the proportion of cells judged by the counter to be cells that were also judged manually to be cells?  It is important to note that there are several factors that can reduce this proportion beyond the program calling things cells that aren't cells.  First, it is possible that a true cell was not marked by the experimenter and the program is finding these cells.  Second, because thresholding reduces the apparent size of an identified cell, and because the marked location of the manual counts is a single pixel, it is possible that the manually marked location of a cell is just outside of the automatically marked location of a cell.  Because of this, the returned value is a lower bound on accuracy.  Thus, if this value is 0.8, the upper bound for false positives is 0.2. 

Currently, I think it should be possible to choose a threshold with >0.8 Hits/Manual Counts and >0.7 for Hits/Automatic Counts.  I have been taking the average of these two and selecting the threshold for which this average is highest.

Lastly, the Otsu method is used to select a threshold that optimally separates signal from noise based upon the distribution of grayscale intensities.  While this generally does a pretty good job, it often seems that a more stringent criterion is helpful to reduce false positives.  In light of this, I think that the Otsu threshold might be the lower bound for what one might choose.


### Creating a composite image
A composite image is made up of  multiple images stitched together side by side (e.g. using ImageJ).  In this way, the chosen parameters are not based upon a single image from one animal.  I personally take 1-2 images from animals in each group and stitch them together. Alternatively, if each image has many cells, it can be easier to do the same thing but take some portion of each image (e.g. 100x100 pixels from each 1000x1000 image).  The composite image should be an 8bit .tif image.  If optimizing multiple channels, the composite image for each channel does not need to overlap with that of the other channel. 


### Creating a mask
Regions of interest can be drawn using ROIdrawer.ipynb.  Alternatively, one can make similar binary masks using ImageJ. In ImageJ:
#1 - Open the image in ImageJ
#2 - Use your preferred selection tool (I like freehand selection tool) to outline the region of interest
#3 - Go to edit > selection > create mask (make sure ROI is white.  invert image if not)
#4 - Save the mask as .tif file


### Manually counting cells from the composite image / Creating a mask
Using ImageJ, the center of each cell in the composite is marked using the multipoint selection tool and this selection is saved as a mask (the center of each cell is set to 255 and all other locations are set to 0).  The mask should be saved as an 8bit .tif image.
*A note on counting cells manually: I typically opt for counting cells that are bright and in focus, while avoiding counting cells that are difficult to make a judgment about.  Additionally, try to center your count locations  in the middle of the brightest portion off your cell.  Because thresholding makes cells smaller in practice, and the location of the manual count is 1 pixel, marking cells in this way maximizes accuracy.


### Folder Structure
For this code to work files should be organized in a specific manner.
Directory_Main (the path of which is to be defined by the user... actual directory name can be whatever you want) should contain the subfolders "Composite", and "ManualCounts" (these names matter).  

Composite and ManualCounts folders should contain .tif images with a single composite image and mask, respectively. 
Example filenames under Composite and ManualCounts: "Study1_Channel_1_Composite.tif","Study1_Channel_1_Composite_Mask.tif"


### Requirements
See README section for package dependencies and conda environment install instructions.

### What type of cells can this count?
Any semi-circular cell that is more or less filled by the fluorescent label will be capable of being counted.  If the cells are only visible by their perimeter than this code as is may not work. In this case, a process should be applied to either a) fill the internal component of the cell after thresholding or b) use smoothing or morphological opening to increase the values of the internal parts of the cell. 

### Viewing an example of how images are being processed
In order to view a single threshold looks like, one can go to the last cell, entitled "Display Result From Single Threshold".  To run this cell of code only the cells up to and including 'Function to Count Cells' must be run


# 1. Load Necessary Packages
The following code loads necessary packages and need not be changed by the user

In [ ]:
%load_ext autoreload
%autoreload 2
import pylab 
import os
import fnmatch
import cv2
import numpy as np
import pandas as pd
import holoviews as hv
import CellCounter_Functions as cc

# 2. User Specifies Parameters and Options for Running
***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.


In [ ]:
#Define directory of working environment (Directory.Main)
dirinfo = {
    'main' : "/Users/zachpennington/Desktop/images"
}

#Set Parameters
params = {
    'diam' : 10, #Average Cell Diameter in pixel units.  Must be integer value.  
    'particle_min' : 0.2 , #After thresholding this serves to erase excessively small points.  Proportion of average cell size permitted.  Average cell size is assumed to be square of diamater for rough approximation.  0.2 has worked well for me 
    'UseWatershed' : True #True/False
}

# 3. Get Directory Information and Display Processing of Image Using Otsu Threshold
Below the user specifies various parameters relevant to analysis.
**Note** that resulting images can be zoomed in on

In [ ]:
%%output size=150
#set output size in line above if too small/large

#Get directory information
dirinfo = cc.optim_getdirinfo(dirinfo)
#Get manual counts and auto count using otsu threshold
images, params, display = cc.optim_getimages(dirinfo,params)
print('Otsu: {x}'.format(x=params['otsu']))
display.cols(2)

# 4. Iterate Through Threshold Values and Count
Data is saved to csv as well as being saved

In [ ]:
data, manual = cc.optim_iterate(images, dirinfo, params, interv=10)
data.to_csv(os.path.join(os.path.normpath(dirinfo['output']), "OptimizataionSummary.csv"))
pd.options.display.max_rows = 255
data

# 5. Display result from single threshold
## Here user can set threshold to any desired value and results will be displayed.

In [ ]:
%%output size=150

#set threshold
params['thresh']=1060

#Display results
count_out = cc.Count(0,"Optim",params,dirinfo,UseROI=False,UseWatershed=params['UseWatershed'])  
display = cc.display_count(count_out)
display.cols(2)